In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [3]:
def calc_logloss(y_true, y_pred):
    k = 1e-5
    y_pred = y_pred.copy()
    y_pred = np.clip(y_pred, a_min=k, a_max=1-k)
    err = - np.mean(y_true * np.log(y_pred) + (1.0 - y_true) * np.log(1.0 - y_pred))
    return err

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [4]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [5]:
X, y

(array([[1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00,
         1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00],
        [1.00e+00, 1.00e+00, 2.00e+00, 1.00e+00, 3.00e+00, 0.00e+00,
         5.00e+00, 1.00e+01, 1.00e+00, 2.00e+00],
        [5.00e+02, 7.00e+02, 7.50e+02, 6.00e+02, 1.45e+03, 8.00e+02,
         1.50e+03, 2.00e+03, 4.50e+02, 1.00e+03],
        [1.00e+00, 1.00e+00, 2.00e+00, 1.00e+00, 2.00e+00, 1.00e+00,
         3.00e+00, 3.00e+00, 1.00e+00, 2.00e+00]]),
 array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.]))

In [6]:
def calc_std_feat(x):
    x[1:] = ((x[1:].T - x[1:].mean(axis=1)) / x[1:].std(axis=1)).T
    return x

In [7]:
x = calc_std_feat(X)

In [8]:
def calc_std_feat(X):
    X_tr = X[1:,].copy()
    X_mean, X_std = X_tr.mean(axis=1), X_tr.std(axis=1)
    X_mean, X_std = X_mean.reshape(X_tr.shape[0], 1), X_std.reshape(X_tr.shape[0], 1)
    X_tr = (X_tr - X_mean)/X_std
    X_tr = np.vstack((np.ones(X_tr.shape[1]), X_tr))
    return X_tr

In [9]:
def sigmoid(z):
    rec = 1 / (1 + np.exp(-z))
    return rec

In [10]:
def eval_model(X, y, iteration, alpha=1e-4, k=1e-5):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    err = []
    w = []
    
    for n_iter in range (1, iteration+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err_new = calc_logloss(y, y_pred)
        W = W - alpha*(1/n * np.dot((y_pred - y), X.T))
        #W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        err.append(err_new)
        w.append(W)
        
        if n_iter > 2 and np.abs(err_new - err[-2]) < k:
            print(n_iter, W, err_new)
            break
    err = np.array(err)
    return err, w

In [11]:
learning_model = [0.1, 0.01, 0.001, 0.0001]

for alpha1 in learning_model:
    W = eval_model(X, y, iteration=1000000, alpha=alpha1)

6236 [ 1.80385313 -5.00023322 -4.03203415 10.50060735] 0.20882355724960694
6193 [ 0.22322901 -2.33314778 -0.81934289  3.73020567] 0.33794925766901673
5495 [ 0.19641082 -0.94005846 -0.0076054   1.43621037] 0.49456917344192275
8140 [ 0.45930427 -0.33157412  0.47217682  1.43686081] 0.6690360538916053


In [12]:
for alpha in [0.1, 0.01, 0.001, 0.0001]:
    err_1, w_1 = eval_model(X=X, y=y, iteration = 1000000, alpha=alpha)

6236 [ 1.80385313 -5.00023322 -4.03203415 10.50060735] 0.20882355724960694
6193 [ 0.22322901 -2.33314778 -0.81934289  3.73020567] 0.33794925766901673
5495 [ 0.19641082 -0.94005846 -0.0076054   1.43621037] 0.49456917344192275
8140 [ 0.45930427 -0.33157412  0.47217682  1.43686081] 0.6690360538916053


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [13]:
def calc_pred_proba(W, X):
    y_pred = 1 /(1+ np.exp(-np.dot(W, X)))
    return y_pred

In [14]:
calc_pred_proba(w_1[-1], X)

array([0.24944723, 0.28765324, 0.70332289, 0.26811923, 0.80633815,
       0.33380776, 0.95597403, 0.95132722, 0.24044325, 0.75149724])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [15]:
def calc_pred(W, X, threshold=0.5):
    y_pred = calc_pred_proba(W, X)
    y_pred = np.where(y_pred > threshold, 1, 0)
    return y_pred

In [16]:
y_pred = calc_pred(w_1[-1], X)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 1, 0, 1])